# Steps for setup

- Install Ollama on the host server
- Run: `sudo systemctl edit ollama.service`
- Append the following under "Anything between here and the comment below will become the new contents of the file"
[Service]
Environment="OLLAMA_HOST=0.0.0.0"
- Run: `sudo systemctl daemon-reload && sudo systemctl restart ollama`
- Connect via the following ssh command: `ssh -L 8080:localhost:11434 jason-server@some_host_Address -p 8000`

In [ ]:
# !pip install requests
# !pip install openai
# !pip install pillow

In [4]:
import requests
#Run this ssh -L 8080:localhost:11434 jason-server@dataarch.ddns.net -p 8000

# Use the specific IP address you provided
OLLAMA_HOST = "http://localhost:8080"

# Endpoint to list available models
url = f"{OLLAMA_HOST}/api/tags"

try:
    response = requests.get(url)
    if response.status_code == 200:
        models = response.json()  # Parse JSON response
        print("Available models:")
        for model in models.get('models', []):
            print(f"- {model.get('name', 'Unknown model')}")
    else:
        print(f"Failed to access Ollama service. Status code: {response.status_code}")
        print(f"Response: {response.text}")

except requests.RequestException as e:
    print(f"An error occurred: {e}")

Available models:
- llama3.3:70b-instruct-q4_0
- llama3.2-vision:11b-instruct-fp16


In [8]:
import requests
from openai import OpenAI
import traceback

try:
    # First, verify server connectivity
    connection_test_url = f"{OLLAMA_HOST}/api/tags"
    print(f"Testing connection to: {connection_test_url}")
    
    connection_test = requests.get(connection_test_url, timeout=5)
    print(f"Connection test status code: {connection_test.status_code}")
    
    # If connection test passes, proceed with OpenAI client
    client = OpenAI(
        base_url = f"{OLLAMA_HOST}/v1",
        api_key='ollama',  # required, but unused
    )

    print("Attempting to create chat completion...")
    response = client.chat.completions.create(
        model="llama3.3:70b-instruct-q4_0",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Who won the world series in 2020?"},
        ]
    )
    
    print("Response received successfully!")
    print(response.choices[0].message.content)

except requests.exceptions.RequestException as req_err:
    print(f"Request Error: {req_err}")
    traceback.print_exc()

except Exception as e:
    print(f"An unexpected error occurred: {e}")
    traceback.print_exc()

Testing connection to: http://localhost:8080/api/tags
Connection test status code: 200
Attempting to create chat completion...
Response received successfully!
The Los Angeles Dodgers won the World Series in 2020, defeating the Tampa Bay Rays in six games (4-2). It was their first championship since 1988.


In [ ]:
import requests
from openai import OpenAI
import traceback
import base64
from PIL import Image
from io import BytesIO
import os

def create_ollama_client(ollama_host):
    """
    Creates an Ollama client with text and image processing capabilities.
    
    Args:
        ollama_host (str): The host URL for the Ollama server
        
    Returns:
        dict: Dictionary containing process_text_query and process_image_query functions
        
    Raises:
        requests.exceptions.RequestException: If server connection fails
        Exception: For other unexpected errors
    """
    def encode_image_to_base64(image_path):
        """
        Convert an image file to base64 string
        """
        try:
            with open(image_path, 'rb') as image_file:
                return base64.b64encode(image_file.read()).decode('utf-8')
        except Exception as e:
            print(f"Error encoding image: {e}")
            return None

    try:
        # First, verify server connectivity
        connection_test_url = f"{ollama_host}/api/tags"
        print(f"Testing connection to: {connection_test_url}")
        
        connection_test = requests.get(connection_test_url, timeout=5)
        print(f"Connection test status code: {connection_test.status_code}")
        
        # If connection test passes, proceed with OpenAI client
        client = OpenAI(
            base_url = f"{ollama_host}/v1",
            api_key='ollama',  # required, but unused
        )

        def process_text_query(prompt, model="llama3.3:70b-instruct-q4_0"):
            """
            Process a text-based query
            
            Args:
                prompt (str): The text prompt to process
                model (str): The model to use for processing
                
            Returns:
                str: The model's response
            """
            print("Attempting to create chat completion...")
            response = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt},
                ]
            )
            return response.choices[0].message.content

        def process_image_query(image_path, prompt, model="llama3.2-vision:11b-instruct-fp16"):
            """
            Process an image-based query
            
            Args:
                image_path (str): Path to the image file
                prompt (str): The text prompt to process along with the image
                model (str): The model to use for processing
                
            Returns:
                str: The model's response
            """
            print("Processing image query...")
            base64_image = encode_image_to_base64(image_path)
            
            if not base64_image:
                return "Error: Could not process image"

            response = client.chat.completions.create(
                model=model,
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {"type": "text", "text": prompt},
                            {
                                "type": "image_url",
                                "image_url": {
                                    "url": f"data:image/jpeg;base64,{base64_image}"
                                }
                            }
                        ]
                    }
                ]
            )
            return response.choices[0].message.content

        return {
            "process_text_query": process_text_query,
            "process_image_query": process_image_query
        }

    except requests.exceptions.RequestException as req_err:
        print(f"Request Error: {req_err}")
        traceback.print_exc()
        raise

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        traceback.print_exc()
        raise


# Example image response

In [24]:
# Use the specific IP address you provided
OLLAMA_HOST = "http://localhost:8080"
ollama_client = create_ollama_client(OLLAMA_HOST)

# Example image query
image_response = ollama_client["process_image_query"](
    os.path.join(os.getcwd(), "tiger.jpg"),
    "What is the breed of the cat is in the image?"
)
print("\nImage Response:")
print(image_response)

Testing connection to: http://localhost:8080/api/tags
Connection test status code: 200
Processing image query...

Image Response:
There does not appear to be an image attached here. However, I can tell you about different types of tabby cats that may look like this if there was an image attached. Tabby cats come in various patterns and colors, but they all share a distinctive M-shaped marking on the forehead that resembles a tiger's stripes.

**There are four known varieties of tabbies:**

1. Classic Tabbies
2. Spotted Tabbies
3. Mackerel (or Ticked) tabbies
4. Blotched Tabbies

The exact breed or type may be difficult to determine without an image, but if there was one attached we could try to identify it with more certainty.


# Example text response

In [20]:
# Use the specific IP address you provided
OLLAMA_HOST = "http://localhost:8080"
ollama_client = create_ollama_client(OLLAMA_HOST)

# Example image query
text_response = ollama_client["process_text_query"](
    "Write a paragraph about the charecteristics of tabby cats in Greek."
)
print(text_response)

Testing connection to: http://localhost:8080/api/tags
Connection test status code: 200
Attempting to create chat completion...
Τα Tabby γάτα είναι μια ποικιλία घरσκής γάτας που φέρει ένα χαρακτηριστικό μοτίβォ στο τρίχωμα τους, το οποίο θυμίζει τη σκούρα και λαμπρόλευκη ξύλινη επιφάνεια ενός παλτού. Αυτό το μοτίβο μπορεί να είναι λευκό με μαύρες ραβδώσεις ή σηмеία, αυξάνοντας την ομορφιά και την εξαίρεση των γατών αυτής της φυλής. Τα Tabby γάτα επίσης έχουν τα χαρκτηριστικά,大的, καστανό-πράσινα μάτια, καθώς και ένα μικρό ρινικό σώμα με μακρύ λαιμό.
